In [20]:
%matplotlib inline 
#有了matplotlib inline 就可以省掉plt.show()了
%load_ext autoreload
%autoreload 2

from __future__ import division
from imports import *
#import pylab #使用pylab模式，可以直接使用numpy、pandas和绘图，而不需要再次引入,会配置为使用你所指定的matplotlib gui后端，还会提供一种更接近于matlab的界面
#%pylab
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle code</button>""", raw=True)
# 允许多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toggle code

## 评估算法
最好的评估办法是利用已经明确知道结果的数据运行生成的算法模型进行验证  

#### 分离训练数据集和评估数据集
通常会将67%的数据作为训练集，将33%的数据作为评估数据集

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
filename = './csvs/user_label_7day_0301.csv'
datasource = pd.read_csv(filename)
#datasource.head(10)
data = datasource[datasource['user_label_7day.fplatformname']=='新浪平台']
# 将数据分为输入数据和输出结果
array = data.values
X = array[:,5:9].astype(np.int64)
Y = array[:,9].astype(np.int64)
Y
test_size = 0.33
seed = 4 # 随机的粒度
X_train,X_test,Y_traing,Y_test = train_test_split(X,Y,test_size=test_size,random_state=seed)
model = LogisticRegression()
model.fit(X_train,Y_traing)
result = model.score(X_test,Y_test)
print("算法评估结果：%.3f%%"% (result*100))

array([ 0, 10,  0, ...,  0, 15,  0], dtype=int64)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

算法评估结果：95.833%


#### K折交叉验证分离
交叉验证是用来验证分类器的性能的一种统计方法  
在样本量不充足的情况下，将数据集A随机分成K个包，每次将其中一个包作为测试集，剩下的k-1个包作为训练集，通常K取3、5、10  
不知道选啥的时候就选这个，然后用10折

In [17]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds,random_state=seed)
model = LogisticRegression()
result = cross_val_score(model,X,Y,cv=kfold)
print("算法评估结果：%.3f%%(%.3f%%)" % (result.mean()*100,result.std()*100))

算法评估结果：95.058%(1.575%)


## 算法评估矩阵 

### 分类算法矩阵
#### 分类准确度 
= 分类正确的样本数/总样本数  
#### 对数损失函数


In [21]:
scoring = 'neg_log_loss'
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
print("Logloss：%.3f%%(%.3f%%)" % (result.mean()*100,result.std()*100))

ValueError: y_true and y_pred contain different number of classes 4, 17. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [ 0  6 10 11]

#### AUC图
ROC和AUC是评价分类器的指标。  
ROC是（Receiver Operating Characteristic Curve），又称感受性曲线。他们都是对同一信号刺激的反应，只不过是在几种不同的判定标准下所得的结果而已。  
AUC是ROC曲线下的面积，通常结余0.5~1.0之间，AUC的值越大，诊断准确性越高。

In [19]:
scoring = 'roc_auc'
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
print("AUC：%.3f%%(%.3f%%)" % (result.mean()*100,result.std()*100))

ValueError: multiclass format is not supported

#### 混淆矩阵（Cnfusion Matrix）
特别适用于监督学习

In [24]:
from sklearn.metrics import confusion_matrix
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test,predicted)
classes = ['0','1']
dataframe = pd.DataFrame(data=matrix,index=classes,columns=classes)
print dataframe

ValueError: Shape of passed values is (10, 10), indices imply (2, 2)

In [ ]:
<img src='./1.png' />

#### 分类报告
在scikit-learn中，Classification_report()方法能够给出精确率（precision）、召回率（recall）、F1值（F1-score）和样本数目（support）  
精确率的公式 $P=\frac{TP}{TP+FP}$  
召回率的公式，$R=\frac{TP}{TP+FN}$  
$F_1=\frac{P+R}2$

In [23]:
from sklearn.metrics import classification_report
report = classification_report(Y_test,predicted)
print report

NameError: name 'predicted' is not defined

## 回归算法矩阵
#### 平均绝对误差 
离差被绝对值化的平均误差

In [ ]:
scoring = 'neg_mean_absolute_error'

#### 均方误差
评价数据变化程度，方差啦就是

In [ ]:
scoring = 'neg_mean_squared_error'

#### 决定系数（$R^2$）
变异能通过回归关系被自变量解释的比例，比如0.8表示回归关系可以解释因变量80%的变异

In [ ]:
scoring = 'r2'